## 1. Import e Setup

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import yaml
import glob

# Aggiungi il path del progetto
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from models.yolo_detector import YOLODetector, visualize_detections
from config import Config

print(f"📂 Project root: {project_root}")
print(f"🎯 Number of classes: {Config.NUM_CLASSES}")
print(f"📦 YOLO model: {Config.YOLO_MODEL}")

## 2. Inizializza il Modello YOLO

**Nota**: I pesi pre-addestrati YOLO saranno scaricati automaticamente e salvati in `checkpoints/pretrained/` invece che nella directory principale del progetto.

In [ ]:
# Crea il detector (usa automaticamente Config.DEVICE)
detector = YOLODetector(
    model_name=Config.YOLO_MODEL,
    pretrained=True,  # Usa i pesi pre-addestrati su COCO
    num_classes=Config.NUM_CLASSES  # LineMOD ha 13 classi
)

# Mostra info sul modello
print(f"\n📊 Model Info:")
for key, value in detector.model_info.items():
    print(f"   {key}: {value}")

## 3. Test su Immagine Singola del Dataset

In [ ]:
# Carica un'immagine dal dataset
dataset_root = Config.DATA_ROOT
sample_img_path = dataset_root / 'data' / '01' / 'rgb' / '0000.png'
sample_gt_path = dataset_root / 'data' / '01' / 'gt.yml'

if sample_img_path.exists():
    # Carica immagine
    image = np.array(Image.open(sample_img_path).convert('RGB'))
    
    # Carica ground truth per confronto
    gt_data = None
    if sample_gt_path.exists():
        with open(sample_gt_path, 'r') as f:
            gt_data = yaml.safe_load(f)
            if 0 in gt_data:  # Image ID 0
                print(f"📋 Ground Truth - {len(gt_data[0])} objects")
                for obj in gt_data[0]:
                    print(f"   Object ID: {obj['obj_id']}, BBox: {obj['obj_bb']}")
    
    print(f"\n📷 Immagine caricata: {sample_img_path}")
    print(f"   Shape: {image.shape}")
    
    # Visualizza immagine originale
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.title("Immagine Originale")
    plt.axis('off')
    plt.show()
else:
    print(f"❌ Immagine non trovata: {sample_img_path}")
    print("   Assicurati di aver scaricato il dataset con il notebook local_dataset_test.ipynb")

## 4. Esegui Detection

In [ ]:
if sample_img_path.exists():
    # Esegui detection
    print("\n🔍 Running object detection...")
    detections = detector.detect_objects(
        image=image,
        conf_threshold=Config.CONF_THRESHOLD
    )
    
    # Mostra risultati
    print(f"\n📦 Detected {len(detections)} objects:")
    for i, det in enumerate(detections):
        print(f"\n   Object {i+1}:")
        print(f"      Class: {det['class_name']} (ID: {det['class_id']})")
        print(f"      Confidence: {det['confidence']:.3f}")
        print(f"      BBox [x1,y1,x2,y2]: {det['bbox']}")

## 5. Visualizza Detections

In [ ]:
if sample_img_path.exists() and len(detections) > 0:
    # Visualizza detections
    vis_image = visualize_detections(image, detections)
    
    # Mostra risultato
    plt.figure(figsize=(12, 12))
    plt.imshow(vis_image)
    plt.title(f"yolo Detection Results ({len(detections)} objects)")
    plt.axis('off')
    plt.show()
elif sample_img_path.exists():
    print("⚠️  Nessun oggetto rilevato. Prova ad abbassare il conf_threshold.")

## 6. Test su Multiple Immagini

In [ ]:
# # Test su più immagini
# import random

# # Trova tutte le immagini RGB
# rgb_images = list((dataset_root / 'data').rglob('rgb/*.png'))

# if len(rgb_images) > 0:
#     # Seleziona 4 immagini random
#     sample_images = random.sample(rgb_images, min(4, len(rgb_images)))
    
#     fig, axes = plt.subplots(2, 2, figsize=(16, 16))
#     axes = axes.flatten()
    
#     for idx, img_path in enumerate(sample_images):
#         # Carica e processa immagine
#         img = np.array(Image.open(img_path).convert('RGB'))
#         dets = detector.detect_objects(img, conf_threshold=0.3)
        
#         # Visualizza
#         vis_img = visualize_detections(img, dets)
#         axes[idx].imshow(vis_img)
#         axes[idx].set_title(f"{img_path.parent.parent.name}/{img_path.name}\n{len(dets)} objects")
#         axes[idx].axis('off')
    
#     plt.tight_layout()
#     plt.show()
# else:
#     print("❌ Nessuna immagine trovata nel dataset")

## 7. Prossimi Passi
- Attualmente YOLO usa pesi pre-addestrati su COCO (80 classi)
- Per LineMOD (13 oggetti specifici) dovrai fare fine-tuning o training da zero
- Il modello baseline serve come confronto per la versione estesa con rotazione